In [7]:
def send_pushover_notification(user_key, app_token, message):
    url = "https://api.pushover.net/1/messages.json"
    data = {
        "token": app_token,
        "user": user_key,
        "message": message
    }
    response = requests.post(url, data=data)
    if response.status_code == 200:
        print("Notification sent successfully.")
    else:
        print(f"Failed to send notification. Status code: {response.status_code}")

with open('../API_Keys/Pushover_USER.txt', 'r') as file:
    USER_KEY = file.read().strip()

with open('../API_Keys/Pushover_APP.txt', 'r') as file:
    APP_TOKEN = file.read().strip()

send_pushover_notification(USER_KEY, APP_TOKEN, "Test")

Notification sent successfully.


In [5]:
import psycopg2
import csv

# Database connection parameters
dbname = "postgres"
user = "postgres"
password = "Passwordd"
host = "localhost"  # or your host if not local
port = "5432"  # default PostgreSQL port

# SQL query to execute
sql_query = """
    SELECT a.*, s.*
    FROM noaa_api a
    JOIN noaa_station_list s
    ON a.station = s.id
    WHERE a.datatype IN ('TMIN', 'TMAX', 'PRCP');
    """

# CSV file path to save the data
csv_file_path = "data_TMINTMAXPRCP_2019_2023.csv"

# Establish connection to the database
try:
    conn = psycopg2.connect(
        dbname=dbname,
        user=user,
        password=password,
        host=host,
        port=port
    )
except Exception as e:
    print("Error in connecting to the database: ", e)
    exit()

# Create a cursor and execute the SQL query
try:
    with conn.cursor() as cur:
        cur.execute(sql_query)

        # Fetch all rows
        rows = cur.fetchall()

        # Write to CSV file
        with open(csv_file_path, 'w', newline='') as csvfile:
            csvwriter = csv.writer(csvfile)
            # Assuming you want to write headers as well
            # Fetch the column names from cursor
            col_names = [desc[0] for desc in cur.description]
            csvwriter.writerow(col_names)
            
            # Write the rows to CSV
            for row in rows:
                csvwriter.writerow(row)

        print(f"Data successfully written to {csv_file_path}")

except Exception as e:
    print("Error in fetching data or writing to CSV: ", e)
finally:
    conn.close()

send_pushover_notification(USER_KEY, APP_TOKEN, "Data download complete")

Data successfully written to data_TMINTMAXPRCP_2019_2023.csv
Notification sent successfully.
